In [2]:
import pandas as pd
import numpy as np

#### N.B. comuni --> province

In [3]:
path = "Data/"
path_foreigners = "Resident foreigners on 1st January - Citizenship/resident_foreigners.csv"
path_resident = "Resident population  on 1st January /resident.csv"

In [4]:
resident_foreigners = pd.read_table(path+path_foreigners)
resident_foreigners = resident_foreigners.drop_duplicates()
resident_foreigners = resident_foreigners.drop("Tipo di indicatore demografico", axis = 1)
resident_foreigners.columns = ["Territorio", "Cittadinanza", "Periodo", "Sesso", "Numero"]
resident_foreigners.head()

,Territorio,Cittadinanza,Periodo,Sesso,Numero
0,Italia,Andorra,2013,maschi,0
1,Italia,Andorra,2014,maschi,2
2,Italia,Andorra,2015,maschi,1
3,Italia,Andorra,2016,maschi,1
4,Italia,Andorra,2017,maschi,1


In [5]:
# Is not useful to keep the info about the ages, since it's an info not available for the foreigners table
resident = pd.read_table(path+path_resident)
resident = resident.drop_duplicates()
resident = resident[resident["Età"] == "totale"]
resident = resident.drop(["Tipo di indicatore demografico", "Stato civile", "Età"], axis = 1)
resident.columns = ["Territorio", "Periodo", "Sesso", "Numero"]
resident.head()

,Territorio,Periodo,Sesso,Numero
179734,Piemonte,2013,maschi,2110576
179735,Piemonte,2014,maschi,2146570
179736,Piemonte,2015,maschi,2140294
179737,Piemonte,2016,maschi,2131892
179738,Piemonte,2017,maschi,2129403


In [6]:
comune_regione = pd.read_table(path+"regioni.csv", sep = "\t")
comune_regione.head()

,Comune,Regione
0,Torino,Piemonte
1,Vercelli,Piemonte
2,Novara,Piemonte
3,Cuneo,Piemonte
4,Asti,Piemonte


In [7]:
regione_zona = pd.read_table(path+"territori.csv", sep = "\t")
regione_zona.head()

,Regione,Zona
0,Abruzzo,Sud
1,Basilicata,Sud
2,Calabria,Sud
3,Campania,Sud
4,Emilia-Romagna,Nord-est


Check if the aggragation is right.. that is: 
- the sum over the municipalities (comuni) conrespond to the aggragation value at the region (regione) level;
- the sum over the regions conrespond to the aggregation value at the zone (nord, sud, etc..) level;
- for each tuple (Territorio, Cittadinanza, Anno) if maschio + femmina = totale.

Check both the resident and the resident_foreigners

In [227]:
regioni = list(set(regione_zona["Regione"]))
zone = list(set(regione_zona["Zona"]))
territori = list(set(resident_foreigners["Territorio"]))
years = list(set(resident_foreigners["Periodo"]))
origine = list(set(resident_foreigners["Cittadinanza"]))

### Resident Foreigners Check

In [77]:
temp = resident_foreigners.copy()
# keep only the municipalities and the region data
territori_1 = list(set(list(comune_regione["Comune"]) + list(comune_regione["Regione"])))
temp = temp.loc[[i in territori_1 for i in temp["Territorio"]]]
# do not consider the Valle d'Aosta / Vallée d'Aoste data since the aggregate and the singular data coincide
temp = temp[temp["Territorio"] != "Valle d'Aosta / Vallée d'Aoste"]
# change the sign to the "total" elements
temp["Numero"][[i in list(set(comune_regione["Regione"])) for i in temp["Territorio"]]] = -temp["Numero"][[i in list(set(comune_regione["Regione"])) for i in temp["Territorio"]]]

regioni = list(set(comune_regione["Regione"]))
comuni = list(set(comune_regione["Comune"]))
# add the regione field, so we can aggregate on it
temp["Regione"] = [comune_regione["Regione"][comune_regione["Comune"] == i].values[0] if i in comuni else i for i in temp["Territorio"]]

# sum all over the same (Regione, Cittadinanza, Periodo, Sesso)
temp['totale'] = temp.groupby(["Regione", "Cittadinanza", "Periodo", "Sesso"])['Numero'].transform(np.sum)

# the sum should always be 0
if len(list(set(temp["totale"]))) == 1 and list(set(temp["totale"]))[0] == 0:
    print("Aggregation maschio + femmina works on 'resident'!")

Aggregation maschio + femmina works on 'resident'!


In [ ]:
'''
for r in regioni:
    print(r)
    for o in origine:   
        comuni = comune_regione[comune_regione["Regione"] == r]["Comune"]
        for y in years:
            temp = 0
            for c in comuni:
                try:
                    temp += resident_foreigners[(resident_foreigners["Territorio"] == c) & (resident_foreigners["Cittadinanza"] == o) & (resident_foreigners["Sesso"] == "totale") & (resident_foreigners["Periodo"] == y)]["Numero"].values
                # Handle missing data
                except ValueError:
                    pass
            if temp != resident_foreigners[(resident_foreigners["Territorio"] == r) & (resident_foreigners["Cittadinanza"] == o) & (resident_foreigners["Sesso"] == "totale") & (resident_foreigners["Periodo"] == y)]["Numero"].values:
                print(temp)
'''

In [86]:
temp = resident_foreigners.copy()
# keep only the municipalities and the region data
territori_1 = list(set(list(regione_zona["Regione"]) + list(regione_zona["Zona"])))
temp = temp.loc[[i in territori_1 for i in temp["Territorio"]]]

# change the sign to the "total" elements
temp["Numero"][[i in list(set(regione_zona["Zona"])) for i in temp["Territorio"]]] = -temp["Numero"][[i in list(set(regione_zona["Zona"])) for i in temp["Territorio"]]]

regioni = list(set(comune_regione["Regione"]))
zone = list(set(regione_zona["Zona"]))

# add the regione field, so we can aggregate on it
temp["Zona"] = [regione_zona["Zona"][regione_zona["Regione"] == i].values[0] if i in regioni else i for i in temp["Territorio"]]

# sum all over the same (Regione, Cittadinanza, Periodo, Sesso)
temp['totale'] = temp.groupby(["Zona", "Cittadinanza", "Periodo", "Sesso"])['Numero'].transform(np.sum)

# the sum should always be 0
if len(list(set(temp["totale"]))) == 1 and list(set(temp["totale"]))[0] == 0:
    print("Aggregation maschio + femmina works on 'resident'!")

Aggregation maschio + femmina works on 'resident'!


In [ ]:
'''
for z in zone:
    print(z)
    for o in origine:   
        regioni = regione_zona[regione_zona["Zona"] == z]["Regione"]
        for y in years:
            temp = 0
            for r in regioni:
                try:
                    temp += resident_foreigners[(resident_foreigners["Territorio"] == r) & (resident_foreigners["Cittadinanza"] == o) & (resident_foreigners["Sesso"] == "totale") & (resident_foreigners["Periodo"] == y)]["Numero"].values
                # Handle missing data
                except ValueError:
                    pass
            if temp != resident_foreigners[(resident_foreigners["Territorio"] == z) & (resident_foreigners["Cittadinanza"] == o) & (resident_foreigners["Sesso"] == "totale") & (resident_foreigners["Periodo"] == y)]["Numero"].values:
                print(temp)
'''

In [12]:
temp = resident_foreigners.copy()
# change the sign to the "total" elements
temp["Numero"][temp[temp["Sesso"] == "totale"].index] = -temp["Numero"][temp[temp["Sesso"] == "totale"].index]
# sum all over the same (Territorio, Cittadinanza, Periodo)
temp['totale'] = temp.groupby(["Territorio", "Cittadinanza", "Periodo"])['Numero'].transform(np.sum)

# the sum should always be 0
if len(list(set(temp["totale"]))) == 1 and list(set(temp["totale"]))[0] == 0:
    print("Aggregation maschio + femmina works on 'resident_foreigners'!")

Aggregation maschio + femmina works on 'resident_foreigners'!


/home/sara/myenv/lib64/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### Resident Check

In [81]:
temp = resident.copy()
# keep only the municipalities and the region data
territori_1 = list(set(list(comune_regione["Comune"]) + list(comune_regione["Regione"])))
temp = temp.loc[[i in territori_1 for i in temp["Territorio"]]]
# do not consider the Valle d'Aosta / Vallée d'Aoste data since the aggregate and the singular data coincide
temp = temp[temp["Territorio"] != "Valle d'Aosta / Vallée d'Aoste"]
# change the sign to the "total" elements
temp["Numero"][[i in list(set(comune_regione["Regione"])) for i in temp["Territorio"]]] = -temp["Numero"][[i in list(set(comune_regione["Regione"])) for i in temp["Territorio"]]]

regioni = list(set(comune_regione["Regione"]))
comuni = list(set(comune_regione["Comune"]))
# add the regione field, so we can aggregate on it
temp["Regione"] = [comune_regione["Regione"][comune_regione["Comune"] == i].values[0] if i in comuni else i for i in temp["Territorio"]]

# sum all over the same (Regione, Periodo, Sesso)
temp['totale'] = temp.groupby(["Regione", "Periodo", "Sesso"])['Numero'].transform(np.sum)

# the sum should always be 0
if len(list(set(temp["totale"]))) == 1 and list(set(temp["totale"]))[0] == 0:
    print("Aggregation maschio + femmina works on 'resident'!")

Aggregation maschio + femmina works on 'resident'!


In [88]:
temp = resident.copy()
# keep only the region and the zone data
territori_1 = list(set(list(regione_zona["Regione"]) + list(regione_zona["Zona"])))
temp = temp.loc[[i in territori_1 for i in temp["Territorio"]]]

# change the sign to the "total" elements
temp["Numero"][[i in list(set(regione_zona["Zona"])) for i in temp["Territorio"]]] = -temp["Numero"][[i in list(set(regione_zona["Zona"])) for i in temp["Territorio"]]]

regioni = list(set(comune_regione["Regione"]))
zone = list(set(regione_zona["Zona"]))

# add the regione field, so we can aggregate on it
temp["Zona"] = [regione_zona["Zona"][regione_zona["Regione"] == i].values[0] if i in regioni else i for i in temp["Territorio"]]

# sum all over the same (Regione, Periodo, Sesso)
temp['totale'] = temp.groupby(["Zona", "Periodo", "Sesso"])['Numero'].transform(np.sum)

# the sum should always be 0
if len(list(set(temp["totale"]))) == 1 and list(set(temp["totale"]))[0] == 0:
    print("Aggregation maschio + femmina works on 'resident'!")

Aggregation maschio + femmina works on 'resident'!


In [11]:
temp = resident.copy()
# change the sign to the "total" elements
temp["Numero"][temp[temp["Sesso"] == "totale"].index] = -temp["Numero"][temp[temp["Sesso"] == "totale"].index]
# sum all over the same (Territorio, Periodo)
temp['totale'] = temp.groupby(["Territorio", "Periodo"])['Numero'].transform(np.sum)

# the sum should always be 0
if len(list(set(temp["totale"]))) == 1 and list(set(temp["totale"]))[0] == 0:
    print("Aggregation maschio + femmina works on 'resident'!")

Aggregation maschio + femmina works on 'resident'!


/home/sara/myenv/lib64/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


The aggragation checks are verified. Thus, it's possible to drop duplication to get the normal form.

### .. to Normal Form

In [115]:
resident_foreigners_norm = resident_foreigners.copy() 

# drop region and zone data
territori_1 = set(list(regione_zona["Regione"]) + (list(regione_zona["Zona"]) + list(["Italia"]))) 
resident_foreigners_norm = resident_foreigners_norm.loc[[i not in territori_1 for i in resident_foreigners_norm["Territorio"]]]
# drop total (mashio+femmina) data
resident_foreigners_norm = resident_foreigners_norm[resident_foreigners_norm["Sesso"] != "totale"]

resident_foreigners_norm.index = range(len(resident_foreigners_norm))

In [119]:
resident_norm = resident.copy() 

# drop region and zone data
territori_1 = set(list(regione_zona["Regione"]) + (list(regione_zona["Zona"]) + list(["Italia"]))) 
resident_norm = resident_norm.loc[[i not in territori_1 for i in resident_norm["Territorio"]]]
# drop total (mashio+femmina) data
resident_norm = resident_norm[resident_norm["Sesso"] != "totale"]

resident_norm.index = range(len(resident_norm))

### Distances

In [235]:
# country specific table
geo_info = pd.read_table(path+"geo_cepii.tsv", sep = "\t")
geo_info = geo_info[['iso3', 'country', 'area', 'dis_int', 'landlocked', 'continent', 'langoff_1', 
                     'langoff_2', 'langoff_3', 'lang20_1', 'lang20_2', 'lang20_3', 'lang20_4', 'lang9_1', 'lang9_2', 
                     'lang9_3', 'lang9_4', 'colonizer1', 'colonizer2', 'colonizer3','colonizer4', 'short_colonizer1', 
                     'short_colonizer2','short_colonizer3']]
# pair of countries distances table
geo_dist = pd.read_table(path+"dist_cepii.tsv", sep = "\t", na_values = {"distw": '.', "distwces": '.'})
geo_dist = geo_dist[['iso_o', 'iso_d', 'contig', 'dist', 'distcap', 'distw', 'distwces']]

In [236]:
geo_info["dis_int"] = [float(geo_info["dis_int"][i].replace(",", ".")) for i in geo_info.index]
geo_info["area"] = geo_info["area"].astype(int)

geo_dist["dist"] = [float(geo_dist["dist"][i].replace(",", ".")) for i in geo_dist.index]
geo_dist["distcap"] = [float(geo_dist["distcap"][i].replace(",", ".")) for i in geo_dist.index]
geo_dist["distw"] = [float(geo_dist["distw"][i].replace(",", ".")) if geo_dist["distw"][i] is not np.nan else np.nan for i in geo_dist.index]
geo_dist["distwces"] = [float(geo_dist["distwces"][i].replace(",", ".")) if geo_dist["distwces"][i] is not np.nan else np.nan for i in geo_dist.index]

In [237]:
geo_info.head()

,iso3,country,area,dis_int,landlocked,continent,langoff_1,langoff_2,langoff_3,lang20_1,...,lang9_2,lang9_3,lang9_4,colonizer1,colonizer2,colonizer3,colonizer4,short_colonizer1,short_colonizer2,short_colonizer3
0,ABW,Aruba,193,5.225315,0,America,Dutch,Spanish,.,Dutch,...,.,.,.,NLD,.,.,.,.,.,.
1,AFG,Afghanistan,652225,303.761400,1,Asia,Persian,.,.,Persian,...,Uzbek,.,.,.,.,.,.,GBR,.,.
2,AGO,Angola,1246700,419.966600,0,Africa,Portuguese,.,.,Portuguese,...,.,.,.,PRT,.,.,.,.,.,.
3,AIA,Anguilla,102,3.798690,0,America,English,.,.,English,...,.,.,.,GBR,.,.,.,.,.,.
4,ALB,Albania,28748,63.773110,0,Europe,Albanian,.,.,Albanian,...,.,.,.,TUR,.,.,.,.,.,.


In [238]:
geo_dist.head()

,iso_o,iso_d,contig,dist,distcap,distw,distwces
0,ABW,ABW,0,5.225315,5.225315,25.09354,23.04723
1,ABW,AFG,0,13257.810000,13257.810000,13168.22000,13166.37000
2,ABW,AGO,0,9516.913000,9516.913000,9587.31600,9584.19300
3,ABW,AIA,0,983.268200,983.268200,976.89740,976.89160
4,ABW,ALB,0,9091.742000,9091.742000,9091.57600,9091.46600


Geo-distances variables:
- iso_o, iso_d: ISO codes in three characters;
- contig: dummy, whether the two countries are contiguous;
- comlang_off: dummy, whether the two countries share the same official language;
- comcol: dummy, whether the two countries have had a common colonizer after 1945;
- colony: dummy, whether the two countries have had a colonial link;
- col45: dummy, whether the two countries have had a colonial relationship after tionship 1945;
- curcol: dummy, whether the two countries are currently in a colonial relationship;
- smctry: dummy, whether the two countries were/are the same country;
- dist: simple distance between the most important cities (in terms of population) of the two countries;
- distcap: simple distance between the most capital cities of the two countries;
- distw, distwces: weighted distances; distance between the largest cities of the two countries - those inter cities distances are weighted by the share of the city in the overall country's population.
